In [1]:
%reload_ext autoreload
%autoreload 2

import os
import pandas as pd

query_path = "/home/wuyinjun/concept_based_retrieval/"

img_caption_file_name= os.path.join(query_path, "prod_hard_negatives/prod_vg_hard_negs_swap_all.csv")

caption_pd = pd.read_csv(img_caption_file_name)

In [2]:
import torch
from transformers import CLIPModel, AutoProcessor


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# processor = ViTImageProcessor.from_pretrained('google/vit-large-patch16-224')
# model = ViTForImageClassification.from_pretrained('google/vit-large-patch16-224').to(device)
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
# processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
raw_processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
# processor =  lambda images: raw_processor(images=images, return_tensors="pt", padding=False, do_resize=False, do_center_crop=False)["pixel_values"]
processor =  lambda images: raw_processor(images=images, return_tensors="pt")["pixel_values"]
text_processor =  lambda text: raw_processor(text=[text], return_tensors="pt", padding=True, truncation=True)
img_processor =  lambda images: raw_processor(images=images, return_tensors="pt")["pixel_values"]
model = model.eval()

/data6/wuyinjun/anaconda3/envs/test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-04 23:31:01.386525: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 23:31:02.610433: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [46]:
caption_ls = list(caption_pd["caption"])
id_ls = list(caption_pd["image_id"])


text_feat_ls = []
visited_id_ls = []
visited_caption_ls = []
with torch.no_grad():

    for idx in range(len(caption_ls)):
        if id_ls[idx] in visited_id_ls:
            continue
        caption = caption_ls[idx]
        inputs = text_processor(caption)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        # print(idx)
        text_features = model.get_text_features(**inputs)
        text_feat_ls.append(text_features.cpu().view(-1))
        visited_id_ls.append(id_ls[idx])
        visited_caption_ls.append(caption)
    
text_feat_tensor = torch.stack(text_feat_ls, dim=0)




In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import Birch

print(len(text_feat_tensor))
clustering = KMeans(n_clusters=200).fit(text_feat_tensor.detach().numpy())
# clustering = Birch(threshold=0.9, n_clusters=None).fit(text_feat_tensor.cpu().numpy())

7405


/home/wuyinjun/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
clustering.labels_

array([138,  31,  44, ...,  48, 153,  22], dtype=int32)

In [ ]:
# print(clustering.labels_)
import numpy as np
unique_label_ids=np.unique(np.array(clustering.labels_))
label_count_ls = [0 for _ in range(len(unique_label_ids))]

for label in clustering.labels_:
    label_count_ls[label] += 1

label_count_tensor = torch.tensor(label_count_ls)

sorted_label_ids = torch.argsort(label_count_tensor, descending=False)


# print(torch.nonzero(label_count_tensor.view(-1) > 1).view(-1))

print(label_count_tensor)

print()



tensor([ 26,  26,  54,  10,  36,  40,  46,  37,  53,  51,  53,  54,  53,  10,
         11,  90,  25,  30,  32,  25,  37,  23,  26,  37,  50,  31,  24,  61,
         21,  60,  77,  64,  70,  70,  37,  15,  27,  38,  28,  20,  33,  25,
         64,  37,  59,  90,  16,  38,  19,  36,  46, 108,  34,  48,  12,  48,
         49,  37,  30,  19,  58,  37,  43,  43,  45,  71,  39,  50,  37,  55,
         52,  45,  50,  35,  38,  30,  62,  19,  36,  19,  24,  42,  35,  21,
         51,  48,  23,  41,  22,  29,  34,  17,  47,  24,  60,  88,  35,  47,
         29,  45,  38,  25,  50,  62,  24,  36,  20,  41,  30,  38,  41,  47,
         22,  38,  31,  24,  54,  46,  18,  18,  31,  13,  55,  17,  34,  45,
         21,  27,  10,  44,  23,  28,  50,  53,  48,  37,  47,  31,  49,  53,
         46,  23,  32,  26,  61,  29,  32,  41,   8,  32,  13,  12,  22,  32,
         41,  38,  14,  32,  57,  52,  18,  23,  17,  39,  35,  30,  45,  42,
          7,  39,  29,  30,  26,  54,  40,  29,  33,  39,  33,  

In [45]:
label_id_ls = torch.nonzero(label_count_tensor.view(-1) == 8).view(-1).tolist()
label_array = torch.tensor(clustering.labels_)
for label_id in label_id_ls:
    sample_ids = torch.nonzero(label_array == label_id).view(-1).tolist()
    for sample_id in sample_ids:
        print(visited_id_ls[sample_id], visited_caption_ls[sample_id])


2360048 feet using a bird cage that has a bird in it. the bird has a feather with a blue patch, and a wing.
2362179 cat on a bird cage with a bird inside. the cat has a tail, and the bird cage has bars.
2362179 a bird in a bird cage with a cat on top of the cage and bars on the cage
2363389 two poles with a bird cage between them. there is a note attached to the bird cage and a bird inside the cage.
2363391 a bird cage with a bird sitting on a bar and a light shining through
2362179 bird, another bird, and cat in a bird cage with a red ring
2368329 a cat is in a cage with a door. the cage has padding and a bowl. there is a wall behind the bowl and the cat's feet are on the padding.
2369274 birds sitting on a branch and a bird in a cage with shrubbery behind it
